In [200]:
import requests
import pandas as pd

class ScriptData:
    def fetch_intraday_data(self, script):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey=B2BT8JP5OMX1745W"
        response = requests.get(url)
        if response.status_code == 200:
            self.data = response.json()
            return self.data
        else:
            return None
        
    def convert_intraday_data(self, script):
        if not hasattr(self, "data"):
            self.fetch_intraday_data(script)
        
        time_series = self.data["Time Series (5min)"]
        data = []
        for timestamp, values in time_series.items():
            open_val = float(values["1. open"])
            high_val = float(values["2. high"])
            low_val = float(values["3. low"])
            close_val = float(values["4. close"])
            volume_val = int(values["5. volume"])
            data.append([timestamp, open_val, high_val, low_val, close_val, volume_val])
        
        df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __contains__(self, key):
        return key in self.data


In [201]:
script_data=ScriptData()

In [302]:
script_data.fetch_intraday_data('AAPL')

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-02-15 20:00:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2023-02-15 20:00:00': {'1. open': '155.4800',
   '2. high': '155.5100',
   '3. low': '155.4800',
   '4. close': '155.4800',
   '5. volume': '14236'},
  '2023-02-15 19:55:00': {'1. open': '155.4800',
   '2. high': '155.5000',
   '3. low': '155.4800',
   '4. close': '155.4900',
   '5. volume': '2574'},
  '2023-02-15 19:50:00': {'1. open': '155.4399',
   '2. high': '155.4399',
   '3. low': '155.4399',
   '4. close': '155.4399',
   '5. volume': '477'},
  '2023-02-15 19:45:00': {'1. open': '155.4900',
   '2. high': '155.4900',
   '3. low': '155.4000',
   '4. close': '155.4000',
   '5. volume': '12929'},
  '2023-02-15 19:40:00': {'1. open': '155.4800',
   '2. high': '155.5000',
   '3. low': '155.4500',
   '4. close': '15

In [303]:
script_data['AAPL']=script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,155.4800,155.5100,155.4800,155.4800,14236
1,2023-02-15 19:55:00,155.4800,155.5000,155.4800,155.4900,2574
2,2023-02-15 19:50:00,155.4399,155.4399,155.4399,155.4399,477
3,2023-02-15 19:45:00,155.4900,155.4900,155.4000,155.4000,12929
4,2023-02-15 19:40:00,155.4800,155.5000,155.4500,155.4900,3467
...,...,...,...,...,...,...
95,2023-02-15 12:00:00,155.3199,155.4300,155.2300,155.2500,825804
96,2023-02-15 11:55:00,155.3100,155.5000,155.2800,155.3100,800875
97,2023-02-15 11:50:00,154.9700,155.3500,154.9200,155.3100,1624024
98,2023-02-15 11:45:00,154.7850,154.9750,154.7600,154.9600,815483


In [304]:
def indicator1(df, timeperiod):
    ma = df['close'].rolling(timeperiod).mean()
    result = pd.DataFrame({'timestamp': df['timestamp'], 'indicator': ma})
    return result

In [305]:
class Strategy(ScriptData):
    def __init__(self, script):
        self.script_data=self.convert_intraday_data(script)
        
        
    def get_script_data(self):
        self.df = self.script_data
        return self.df
    
    def compute_indicator(self, timeperiod):
        self.indicator_data = indicator1(self.df, timeperiod)
        return self.indicator_data

    def get_signals(self):
        signals = pd.DataFrame({"timestamp": self.df["timestamp"], "signal": "NO_SIGNAL"})
        for i in range(1, len(self.df)):
            if self.indicator_data.loc[i,'indicator'] > self.df["close"].iloc[i] and self.indicator_data.loc[i-1,'indicator'] <= self.df["close"].iloc[i-1]:
                signals.loc[i, "signal"] = "BUY"
            elif self.indicator_data.loc[i,'indicator'] < self.df["close"].iloc[i] and self.indicator_data.loc[i-1,'indicator'] >= self.df["close"].iloc[i-1]:
                signals.loc[i, "signal"] = "SELL"
        self.signals = signals[signals["signal"].isin(["BUY", "SELL"])]
        return self.signals

In [306]:
strategy=Strategy("AAPL")


In [307]:
strategy.get_script_data()

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,155.4800,155.5100,155.4800,155.4800,14236
1,2023-02-15 19:55:00,155.4800,155.5000,155.4800,155.4900,2574
2,2023-02-15 19:50:00,155.4399,155.4399,155.4399,155.4399,477
3,2023-02-15 19:45:00,155.4900,155.4900,155.4000,155.4000,12929
4,2023-02-15 19:40:00,155.4800,155.5000,155.4500,155.4900,3467
...,...,...,...,...,...,...
95,2023-02-15 12:00:00,155.3199,155.4300,155.2300,155.2500,825804
96,2023-02-15 11:55:00,155.3100,155.5000,155.2800,155.3100,800875
97,2023-02-15 11:50:00,154.9700,155.3500,154.9200,155.3100,1624024
98,2023-02-15 11:45:00,154.7850,154.9750,154.7600,154.9600,815483


In [308]:
strategy.compute_indicator(5)

,timestamp,indicator
0,2023-02-15 20:00:00,NaN
1,2023-02-15 19:55:00,NaN
2,2023-02-15 19:50:00,NaN
3,2023-02-15 19:45:00,NaN
4,2023-02-15 19:40:00,155.45998
...,...,...
95,2023-02-15 12:00:00,155.00414
96,2023-02-15 11:55:00,155.09014
97,2023-02-15 11:50:00,155.17414
98,2023-02-15 11:45:00,155.19500


In [309]:
strategy.get_signals()

,timestamp,signal
8,2023-02-15 19:20:00,BUY
9,2023-02-15 19:15:00,SELL
10,2023-02-15 19:10:00,BUY
14,2023-02-15 18:50:00,SELL
16,2023-02-15 18:35:00,BUY
22,2023-02-15 18:05:00,SELL
24,2023-02-15 17:55:00,BUY
25,2023-02-15 17:50:00,SELL
27,2023-02-15 17:40:00,BUY
28,2023-02-15 17:35:00,SELL
